In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id='17EEWVSQbVsphNZilxDokFUAs921ZZAwe'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('data')


In [2]:
!pip install pyLDAvis
!pip install hlda

     |████████████████████████████████| 1.6MB 4.6MB/s 
     |████████████████████████████████| 552kB 29.8MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=e9ed69899dd9581652c73edd0ab971c19d29d559ff6b06b61586fc9bbe1c96fc
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=043d6a79d63004223bf0da21556bceca956266204e747649eb9f9d2eace66211
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy
  Created wheel for hlda: filename=hlda-0.2-cp36-none-any.whl size=5034 sha256=65aa9ceecc1f1287d8dbeae1a59589b8eacd2b22002c31b3af804dfd828e9287
  Stored in directory: /root/.cache/pip/wheels/c0/96/0d/79c6a4e965efa74c4541cc833dda92244d5477e4831a5f0f7e
Successfully built hlda


In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import nltk; nltk.download('stopwords')
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

letters = ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]
other = ["wa","ha","one","two","id","re","http","com","mr","image","photo","caption","don","sen","pic","co",
             "source","watch","play","duration","video","momentjs","getty","images","newsletter", "story", "go", "like", "say",
             "will", "just", "today", "can", "year", "make", "view", "read",'tell',"also","thing","things","would"]
stop_words.extend(letters)
stop_words.extend(other)

In [5]:
df = pd.read_csv('data', dtype = {"article_text": str, "article_id":str, "article_title": str, "article_keywords":str, 'article_published_on':str}, parse_dates = ['article_published_on'])

In [6]:
data = df.article_text.values.tolist()
data = [re.sub('\S*@\S*\s?', '', str(sent)).encode('utf-8') for sent in data]
data = [re.sub('\\\\x[a-z0-9][a-z0-9][a-z0-9]?', ' ', str(sent)) for sent in data]
data = [re.sub('\s+', ' ', str(sent)) for sent in data]
data = [re.sub("\'", "", str(sent)) for sent in data]

<input>:2: DeprecationWarning: invalid escape sequence \S
<input>:2: DeprecationWarning: invalid escape sequence \S
<input>:2: DeprecationWarning: invalid escape sequence \S
<input>:2: DeprecationWarning: invalid escape sequence \S
<input>:2: DeprecationWarning: invalid escape sequence \S
<input>:2: DeprecationWarning: invalid escape sequence \S
<input>:2: DeprecationWarning: invalid escape sequence \S
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:2: DeprecationWarning: invalid escape sequence \S
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:2: DeprecationWarning: invalid escape sequence \S
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:2: DeprecationWarning: invalid escape sequence \S
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:2: DeprecationWarning: invalid escape sequence \S
<input>:4: DeprecationWarning: invalid escape sequence \s
<input>:2: DeprecationWarning: invalid escape sequence \S
<input>:4: Dep

In [7]:
len(data)

515

In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [9]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [10]:
data_words_nostops = remove_stopwords(data_words)

In [11]:
bigram = gensim.models.Phrases(data_words_nostops, min_count=5, threshold=100) # higher threshold fewer phrases.
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [12]:
def make_bigrams(texts, bigram_model = bigram_mod):
    return [bigram_model[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
data_words_bigrams = make_bigrams(data_words_nostops)

In [14]:
len(data_words_bigrams)

515

In [15]:
nlp = spacy.load('en', disable=['parser', 'ner'])

In [16]:
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [17]:
data_lemmatized = remove_stopwords(data_lemmatized)
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]

In [18]:
scr=[]
for j in range(5,35,5):
  lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=j,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
  coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  scr.append(coherence_lda)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

KeyboardInterrupt: ignored

In [ ]:
plt.plot(range(10,35,5),scr)
plt.legend()
plt.title("Coherency Vs Number of Clusters")
plt.xlabel("Coherency")
plt.ylabel("Number of Clusters")
plt.show()

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


In [ ]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

In [ ]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
new_doc = pd.read_csv('data',sep=',', error_bad_lines=False,encoding="latin-1")
new_doc=new_doc.drop('Unnamed: 0',axis=1).dropna()
new_doc.reset_index(drop=True, inplace=True)
new_doc.head()

In [ ]:
data = new_doc.article_text.values.tolist()
data = [re.sub('\S*@\S*\s?', '', str(sent)).encode('utf-8') for sent in data]
data = [re.sub('\\\\x[a-z0-9][a-z0-9][a-z0-9]?', ' ', str(sent)) for sent in data]
data = [re.sub('\s+', ' ', str(sent)) for sent in data]
data = [str(sent).replace('\\n', '') for sent in data]
data = [str(sent).replace('\\r', '') for sent in data]
data = [re.sub("\'", "", str(sent)) for sent in data]

In [ ]:
politifcat_topics = []
for item in data:
  sentence= (item[1:])
  data_words_sent= gensim.utils.simple_preprocess(str(sentence), deacc=True)
  data_words_sent = [data_words_sent]
  bigram_sent = gensim.models.Phrases(data_words_sent, min_count=5, threshold=100)
  bigram_mod_sent = gensim.models.phrases.Phraser(bigram_sent)
  data_words_nostops_sent = remove_stopwords(data_words_sent)
  data_words_bigrams_sent = make_bigrams(data_words_nostops_sent, bigram_mod_sent)
  data_lemmatized_sent = lemmatization(data_words_bigrams_sent, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
  data_lemmatized_sent = remove_stopwords(data_lemmatized_sent)
  id2word_sent = corpora.Dictionary(data_lemmatized_sent)
  corpus_sent = [id2word_sent.doc2bow(data_sent) for data_sent in data_lemmatized_sent]
  topics_in_sent=lda_model.get_document_topics(corpus_sent, minimum_probability=0.0)
  for i in topics_in_sent:
    politifcat_topics.append(i)

In [ ]:
df_results=pd.DataFrame(politifcat_topics)
df_results

In [ ]:
df_results_2=df_results
for ind, rows in df_results.iterrows():
  for indj,j in rows:
    df_results_2[indj][ind]=j

In [ ]:
nums=[0,1,30,100,250,375]
fig=plt.figure(figsize=(15, 10))
cnt=0
for i in nums:
  cnt+=1
  fig.add_subplot(2, 3, cnt)  
  plt.bar(range(5,35,5),df_results_2.loc[i]*100)
  plt.xlabel("topic number")
  plt.ylabel("% of topic")
  plt.title("topic distribution for article "+str(i))

In [ ]:
#print(df_results_2[0])
for i in range(0,5):
  df_results_2[i]=pd.to_numeric(df_results_2[i])
top=df_results_2.apply(lambda x: x.astype(float).argmax(), axis=1)

In [ ]:
df['cluster']=top

In [ ]:
df[['article_id','cluster']].to_csv("results_basic_lda.csv",index=False)

In [ ]:
plt.hist(top)
plt.xlim(0,20)
plt.xlabel("Topic Number")
plt.ylabel("Number of Documents")
plt.title("Distribution of counts of Documents over Topic Number")

In [ ]:
new_doc['cluster']=top

In [ ]:
import matplotlib.cm as cm
def plot_tsne_pca(data, labels):
    max_label = max(labels)
    max_items = np.random.choice(range(data.shape[0]), size=3000)
    
    pca = PCA(n_components=2).fit_transform(data[max_items,:].todense())
    tsne = TSNE().fit_transform(PCA(n_components=50).fit_transform(data[max_items,:].todense()))
    
    
    idx = np.random.choice(range(pca.shape[0]),size=515, replace=False)
    label_subset = labels[max_items]
    label_subset = [cm.hsv(i/max_label) for i in label_subset[idx]]
    
    f, ax = plt.subplots(1, 2, figsize=(14, 6))
    
    ax[0].scatter(pca[idx, 0], pca[idx, 1], c=label_subset)
    ax[0].set_title('PCA Cluster Plot')
    
    ax[1].scatter(tsne[idx, 0], tsne[idx, 1], c=label_subset)
    ax[1].set_title('TSNE Cluster Plot')
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
tfidf = TfidfVectorizer(
    min_df = 5,
    max_df = 0.95,
    max_features = 8000,
    stop_words = 'english'
)
tfidf.fit(data)
text = tfidf.transform(data)
plot_tsne_pca(text, top.values)

In [ ]:
top.values